<a href="https://colab.research.google.com/github/Butanium/nnterp/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo: nnterp Features Showcase

This notebook demonstrates the key features of `nnterp`, which aims to offer a unified interface for all transformer models and give best `NNsight` practices for LLMs in everyone's hands.

In [1]:
try:
  import google.colab
  !pip install git+https://github.com/Butanium/nnterp.git
except:
  pass

## 1. Standardized Interface

Similar to [`transformer_lens`](https://github.com/TransformerLensOrg/TransformerLens), `nnterp` provides a standardized interface for all transformer models.
The main difference is that `nnterp` still uses the huggingface implementation under the hood through `NNsight`, while transformer_lens uses its own implementation of the transformer architecture. However, each transformer implementation has its own quirks, such that `transformer_lens` is not able to support all models, and can sometimes have significant difference with the huggingface implementation.

Note that `nnterp` doesn't support all models either, since `NNsight` itself doesn't support all architectures. Additionally, because different models use different naming conventions, `nnterp` doesn't support all HuggingFace models, but it does support a good portion of them. When a model is loaded in `nnterp`, automatic tests are performed to verify that the model has been correctly renamed and that `nnterp`'s hooks return the expected shapes. This means that even if an architecture hasn't been officially tested, the simple fact that it loads successfully indicates it's probably working correctly.

The way it's implemented is based on the `NNsight` built-in renaming feature, to make all models look like the llama naming convention, without having to write `model.model`, namely:
```ocaml
StandardizedTransformer
├── embed_tokens
├── layers
│   ├── self_attn
│   └── mlp
├── ln_final
└── lm_head
```

In [2]:
from transformers import AutoModelForCausalLM

print(AutoModelForCausalLM.from_pretrained("Maykeye/TinyLLama-v0"))
print(AutoModelForCausalLM.from_pretrained("gpt2"))

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 64, padding_idx=0)
    (layers): ModuleList(
      (0-7): 8 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=64, out_features=64, bias=False)
          (k_proj): Linear(in_features=64, out_features=64, bias=False)
          (v_proj): Linear(in_features=64, out_features=64, bias=False)
          (o_proj): Linear(in_features=64, out_features=64, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=64, out_features=256, bias=False)
          (up_proj): Linear(in_features=64, out_features=256, bias=False)
          (down_proj): Linear(in_features=256, out_features=64, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((64,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((64,), eps=1e-06)
      )
    )
    (norm): LlamaRMSNorm((64,), eps=1e-06)
    (rotary_emb): LlamaRotaryEmbeddi

As you can see, the naming scheme of gpt2 is different from the llama naming convention.
A simple way to fix this is to use the `rename` feature of `NNsight` to rename the gpt2 modules to the llama naming convention.

In [3]:
from nnsight import LanguageModel

model = LanguageModel(
    "gpt2",
    rename=dict(transformer="model", h="layers", ln_f="ln_final", attn="self_attn"),
)
print(model)
# Access the attn module as if it was a llama model
print(model.model.layers[0].self_attn)

GPT2LMHeadModel(
  (model/transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (layers/h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (self_attn/attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_final/ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
  (generator): Generator(
    (streamer): Streamer()
  )
)
GPT2Attention(
  (c_att

You can see the that renamed modules are displayed like `(new_name)/old_name`. However, many models family have their own naming convention, `nnterp` has a global renaming scheme that should transform any model to the llama naming convention. The easiest way to use it is to load your model using the `StandardizedTransformer` class that inherits from `nnsight.LanguageModel`.

In [ ]:
from nnterp import StandardizedTransformer

# You will see the `layers` module printed two times, it'll be explained later.
# Note: We use enable_attention_probs=True to enable attention pattern access
nnterp_gpt2 = StandardizedTransformer("gpt2", enable_attention_probs=True)
print(nnterp_gpt2)
# StandardizedTransformer also use `device_map="auto"` by default:
nnterp_gpt2.dispatch()
print(nnterp_gpt2.model.device)

Great! But I can see you at the back of the classroom, asking yourself:
> "Why would you create a package that just pass the right dict to the `NNsight` `rename` feature?"

And actually, I'm glad you asked! `StandardizedTransformer` and `nnterp` have a lot of other features, so bear with me!

## 2. Accessing Modules I/O
With `NNsight`, the most robust way to set the residual stream after layer 1 to be the residual stream after layer 0 for a LLama-like model would be:

In [5]:
from transformers import __version__ as TRANSFORMERS_VERSION
from packaging.version import parse
llama = LanguageModel("Maykeye/TinyLLama-v0")
# code for transformer "<4.54"
is_old_transformers = parse(TRANSFORMERS_VERSION) < parse("4.54")
if is_old_transformers:
    with llama.trace("hello"):
        llama.model.layers[1].output = (
            llama.model.layers[0].output[0],
            *llama.model.layers[1].output[1:],
        )
else:
    with llama.trace("hello"):
        llama.model.layers[1].output = llama.model.layers[0].output


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


Note that the following can cause issues:

In [6]:
with llama.trace("hello"):
    # can't do this because .output is a tuple

    # Can cause errors with gradient computation
    if is_old_transformers:
        # llama.model.layers[1].output[0] = llama.model.layers[0].output[0]
        llama.model.layers[1].output[0][:] = llama.model.layers[0].output[0]
    else:
        llama.model.layers[1].output[:] = llama.model.layers[0].output

if is_old_transformers:
    with llama.trace("hello"):
        # Can cause errors with opt if you do this at its last layer (thanks pytest)
        llama.model.layers[1].output = (llama.model.layers[0].output[0],)

`nnterp` makes this much cleaner:

In [7]:
# the version of transformers does not matter, the tuple vs not tuple stuff is handled internally
# First, you can access layer inputs and outputs directly:
with nnterp_gpt2.trace("hello"):
    # Access layer 5's output
    layer_5_output = nnterp_gpt2.layers_output[5]
    # Set layer 10's output to be layer 5's output
    nnterp_gpt2.layers_output[10] = layer_5_output

# You can also access attention and MLP outputs:
with nnterp_gpt2.trace("hello"):
    attn_output = nnterp_gpt2.attentions_output[3]
    mlp_output = nnterp_gpt2.mlps_output[3]

## 3. `nnterp` Guarantees


When designing, `nnterp` I was very worried about silent failures, where you load a model, and then get an unexpected failure in your code downstream, or worst, it doesn't fail but give you fake results. To avoid this, when you load an `nnterp` model, a series of fast tests are run to ensure that:
- The model has been correctly renamed
- The model module output are of the expected shape
- Attention probabilities have the right shape, sum to 1, and changing them changes the output

This comes with the trade-off that `nnterp` will dispatch your model when you load it, which can be annoying if you don't want to load the model's weights. Also to be able to access the attention probabibilties, `nnterp` loads your model with the `eager` attention implementation, which can be slower than the default hf implementation. If you don't need the attention probabilities, you can force to use the default hf implementation / another one by passing `attn_implementation=None` or `attn_implementation="your_implementation"`.

What `nnterp` can NOT guarantee:
- The attention probabilities won't be modified by the model before being multiplied by the values. To ensure this, you can check `model.attention_probabilities.print_source()` (preferably in a notebook for markdown display) to understand where the attention probabilities are computed.
- Huggingface's transformers sheringan w

If youe model is not properly renamed, you can pass a `RenameConfig` to the `nnterp` constructor to rename the model. See more in the advanced usage section of this demo.

On top of that, before releasing a new version of `nnterp`, a series of tests covering most architectures are performed. When you load a model, `nnterp` will check if tests were run for your `nnsight` and `transformers` versions, and will check the tests results for the class of your model. I chose to include the tests in the `nnterp` package, so that if your model architecture has not been tested / you use a different version of `nnsight` or `transformers`, you can run `python -m nnterp run_tests --model-names foo bar --class-names LlamaForCausalLM` to run the tests for your model. `--class-names` allow you to run the tests on a toy model of the same class as your model to make it cheaper and faster.

## 4. Attention Probabilities

For models that support it, you can access attention probabilities directly. You can check if a model supports it by calling `model.supports_attention_probabilities`.

In [8]:
import torch as th

nnterp_gpt2.tokenizer.padding_side = (
    "left"  # ensure left padding for easy access to the first token
)

with th.no_grad():
    with nnterp_gpt2.trace("The cat sat on the mat"):
        # Access attention probabilities for layer 5
        attn_probs_l2 = nnterp_gpt2.attention_probabilities[2].save()
        attn_probs = nnterp_gpt2.attention_probabilities[5].save()
        print(
            f"Attention probs shape will be: (batch, heads, seq_len, seq_len): {attn_probs.shape}"
        )
        # knock out the attention to the first token
        attn_probs[:, :, :, 0] = 0
        attn_probs /= attn_probs.sum(dim=-1, keepdim=True)
        corr_logits = nnterp_gpt2.logits.save()
    with nnterp_gpt2.trace("The cat sat on the mat"):
        baseline_logits = nnterp_gpt2.logits.save()

assert not th.allclose(corr_logits, baseline_logits)

sums = attn_probs_l2.sum(dim=-1)
# last dimension is the attention of token i to all other tokens, so should sum to 1
assert th.allclose(sums, th.ones_like(sums))

Attention probs shape will be: (batch, heads, seq_len, seq_len): torch.Size([1, 12, 6, 6])


Under the hood this uses the new tracing system implemented in `NNsight v0.5` which allow to access most model intermediate variables during the forward pass. This means that if the `transformers` implementation were to change, this could break or give unexpected results, so it is recommended to use one of the tested versions of `transformers` and to check that the attention probabilities hook makes sense by calling `model.attention_probabilities.print_source()` if you want to use a different version of `transformers` / a architecture that has not been tested.

In [9]:
nnterp_gpt2.attention_probabilities.print_source()  # pretty markdown display in a notebook

## Accessing attention probabilities from:
```py
model.transformer.h.0.attn.attention_interface_0.module_attn_dropout_0:

    ....
            attn_weights = attn_weights + causal_mask
    
        attn_weights = nn.functional.softmax(attn_weights, dim=-1)
    
        # Downcast (if necessary) back to V's dtype (if in mixed-precision) -- No-Op otherwise
        attn_weights = attn_weights.type(value.dtype)
    --> attn_weights = module.attn_dropout(attn_weights) <--
    
        # Mask heads if we want to
        if head_mask is not None:
            attn_weights = attn_weights * head_mask
    
        attn_output = torch.matmul(attn_weights, value)
    ....
```

## Full module source:
```py
                             * def eager_attention_forward(module, query, key, value, attention_mask, head_mask=None, **kwargs):
 key_transpose_0         ->  0     attn_weights = torch.matmul(query, key.transpose(-1, -2))
 torch_matmul_0          ->  +     ...
                             1 
                             2     if module.scale_attn_weights:
 torch_full_0            ->  3         attn_weights = attn_weights / torch.full(
 value_size_0            ->  4             [], value.size(-1) ** 0.5, dtype=attn_weights.dtype, device=attn_weights.device
                             5         )
                             6 
                             7     # Layer-wise attention scaling
                             8     if module.scale_attn_by_inverse_layer_idx:
 float_0                 ->  9         attn_weights = attn_weights / float(module.layer_idx + 1)
                            10 
                            11     if not module.is_cross_attention:
                            12         # if only "normal" attention layer implements causal mask
 query_size_0            -> 13         query_length, key_length = query.size(-2), key.size(-2)
 key_size_0              ->  +         ...
                            14         causal_mask = module.bias[:, :, key_length - query_length : key_length, :key_length]
 torch_finfo_0           -> 15         mask_value = torch.finfo(attn_weights.dtype).min
                            16         # Need to be a tensor, otherwise we get error: `RuntimeError: expected scalar type float but found double`.
                            17         # Need to be on the same device, otherwise `RuntimeError: ..., x and y to be on the same device`
 torch_full_1            -> 18         mask_value = torch.full([], mask_value, dtype=attn_weights.dtype, device=attn_weights.device)
 attn_weights_to_0       -> 19         attn_weights = torch.where(causal_mask, attn_weights.to(attn_weights.dtype), mask_value)
 torch_where_0           ->  +         ...
                            20 
                            21     if attention_mask is not None:
                            22         # Apply the attention mask
                            23         causal_mask = attention_mask[:, :, :, : key.shape[-2]]
                            24         attn_weights = attn_weights + causal_mask
                            25 
 nn_functional_softmax_0 -> 26     attn_weights = nn.functional.softmax(attn_weights, dim=-1)
                            27 
                            28     # Downcast (if necessary) back to V's dtype (if in mixed-precision) -- No-Op otherwise
 attn_weights_type_0     -> 29     attn_weights = attn_weights.type(value.dtype)
 module_attn_dropout_0   -> 30     attn_weights = module.attn_dropout(attn_weights)
                            31 
                            32     # Mask heads if we want to
                            33     if head_mask is not None:
                            34         attn_weights = attn_weights * head_mask
                            35 
 torch_matmul_1          -> 36     attn_output = torch.matmul(attn_weights, value)
 attn_output_transpose_0 -> 37     attn_output = attn_output.transpose(1, 2)
                            38 
                            39     return attn_output, attn_weights
                            40 
```

## 5. Builtin interventions

`StandardizedTransformer` also provides convenient methods for common operations:

In [10]:
import torch as th

# Project hidden states to vocabulary using the unembed norm and lm_head
with nnterp_gpt2.trace("The capital of France is"):
    hidden = nnterp_gpt2.layers_output[5]
    logits = nnterp_gpt2.project_on_vocab(hidden)

# Skip layers entirely
with nnterp_gpt2.trace("Hello world"):
    # Skip layer 1
    nnterp_gpt2.skip_layer(1)
    # Skip layers 2 through 3 (inclusive)
    nnterp_gpt2.skip_layers(2, 3)

# This is useful if you want to start at a later layer than the first one
with nnterp_gpt2.trace("Hello world") as tracer:
    layer_6_out = nnterp_gpt2.layers_output[6].save()
    tracer.stop()  # avoid computations after layer 6

with nnterp_gpt2.trace("Hello world"):
    nnterp_gpt2.skip_layers(0, 6, skip_with=layer_6_out)
    half_half_logits = nnterp_gpt2.logits.save()

with nnterp_gpt2.trace("Hello world"):
    vanilla_logits = nnterp_gpt2.logits.save()

assert th.allclose(vanilla_logits, half_half_logits)  # they should be the same

# Direct steering
steering_vector = th.randn(768)  # gpt2 hidden size
with nnterp_gpt2.trace("The weather today is"):
    nnterp_gpt2.steer(layers=[1, 3], steering_vector=steering_vector, factor=0.5)

## 6. Specific Token Activation Collection

`nnterp` provides utilities for collecting activations efficiently:

In [11]:
from nnterp.nnsight_utils import (
    get_token_activations,
    collect_token_activations_batched,
)

# Collect activations for specific tokens
prompts = ["The capital of France is", "The weather today is"]
with nnterp_gpt2.trace(prompts) as tracer:
    # Get last token activations for all layers
    activations = get_token_activations(nnterp_gpt2, prompts, idx=-1, tracer=tracer)
    # activations shape: (num_layers, batch_size, hidden_size)

# For large datasets, use batched collection
large_prompts = ["Sample text " + str(i) for i in range(100)]
batch_activations = collect_token_activations_batched(
    nnterp_gpt2,
    large_prompts,
    batch_size=16,
    layers=[3, 9, 11],  # Only collect specific layers, default is all layers
    idx=-1,  # Last token (default)
)
print(f"Batched activations shape: {batch_activations.shape}")

Batched activations shape: torch.Size([3, 100, 768])


## 7. Prompt Utilities

`nnterp` provides utilities for working with prompts and tracking probabilities of first tokens of certain strings. It tracks both the first token of "string" and " string".

You can provide multiple string per category, the probabilities returned will be the sum of the probabilities of all the first tokens of the strings.

In [12]:
from nnterp.prompt_utils import Prompt, run_prompts

# Create prompts with target tokens to track
prompt1 = Prompt.from_strings(
    "The capital of France (not England or Spain) is",
    {
        "target": "Paris",
        "traps": ["London", "Madrid"],
        "longstring": "the country of France",
    },
    nnterp_gpt2.tokenizer,
)
for name, tokens in prompt1.target_tokens.items():
    print(f"{name}: {nnterp_gpt2.tokenizer.convert_ids_to_tokens(tokens)}")

prompt2 = Prompt.from_strings(
    "The largest planet (not Earth or Neptune) is",
    {"target": "Jupiter", "traps": ["Earth", "Neptune"], "longstring": "Palace planet"},
    nnterp_gpt2.tokenizer,
)
for name, tokens in prompt2.target_tokens.items():
    print(f"{name}: {nnterp_gpt2.tokenizer.convert_ids_to_tokens(tokens)}")

# Run prompts and get probabilities for target tokens
results = run_prompts(nnterp_gpt2, [prompt1, prompt2], batch_size=2)
print("Target token probabilities:")
for target, probs in results.items():
    print(f"  {target}: shape {probs.shape}")

target: ['Paris', 'ĠParis']
traps: ['London', 'ĠLondon', 'Mad', 'ĠMadrid']
longstring: ['the', 'Ġthe']
target: ['J', 'ĠJupiter']
traps: ['Earth', 'ĠEarth', 'Ne', 'ĠNeptune']
longstring: ['Pal', 'ĠPalace']


Running prompts:   0%|          | 0/1 [00:00<?, ?it/s]

Target token probabilities:
  target: shape torch.Size([2, 1])
  traps: shape torch.Size([2, 1])
  longstring: shape torch.Size([2, 1])


## 8. Interventions

`nnterp` provides several intervention methods inspired by mechanistic interpretability research:

In [13]:
from nnterp.interventions import (
    logit_lens,
    patchscope_lens,
    TargetPrompt,
    repeat_prompt,
    steer,
)

# Logit Lens: See predictions at each layer
prompts = ["The capital of France is", "The sun rises in the"]
probs = logit_lens(nnterp_gpt2, prompts)
print(f"Logit lens output shape: {probs.shape}")  # (batch, layers, vocab)

# Patchscope: Replace activations from one context into another
source_prompts = ["Paris is beautiful", "London is foggy"]
custom_target_prompt = TargetPrompt("city: Paris\nfood: croissant\n?", -1)
target_prompt = repeat_prompt()  # Creates a repetition task
custom_repeat_prompt = repeat_prompt(
    words=["car", "cross", "azdrfa"],
    rel=":",
    sep="\n\n",
    placeholder="*",
)
print(f"repeat_prompt: {custom_repeat_prompt}")
print(f"custom_repeat_prompt: {custom_repeat_prompt}")
patchscope_probs = patchscope_lens(
    nnterp_gpt2, source_prompts=source_prompts, target_patch_prompts=target_prompt
)
print(f"patchscope_probs: {patchscope_probs.shape}")

# Steering with intervention function
with nnterp_gpt2.trace("The weather is"):
    steer(nnterp_gpt2, layers=[5, 10], steering_vector=steering_vector)

Logit lens output shape: torch.Size([2, 12, 50257])
repeat_prompt: TargetPrompt(prompt='car:car\n\ncross:cross\n\nazdrfa:azdrfa\n\n*', index_to_patch=-1)
custom_repeat_prompt: TargetPrompt(prompt='car:car\n\ncross:cross\n\nazdrfa:azdrfa\n\n*', index_to_patch=-1)
patchscope_probs: torch.Size([2, 12, 50257])


You can use a combination of run_prompts and interventions to get the probabilities of certain tokens according to your custom intervention.

In [14]:
demo_model = nnterp_gpt2
# uncomment if you have a GPU for cooler results
# demo_model = StandardizedTransformer("google/gemma-2-2b")

prompts_str = [
    "The translation of 'car' in French is",
    "The translation of 'cat' in Spanish is",
]
tokens = [
    {"target": ["voiture", "bagnole"], "english": "car", "format": "'"},
    {"target": ["gato", "minino"], "english": "cat", "format": "'"},
]
prompts = [
    Prompt.from_strings(prompt, tokens, demo_model.tokenizer)
    for prompt, tokens in zip(prompts_str, tokens)
]
results = run_prompts(demo_model, prompts, batch_size=2, get_probs_func=logit_lens)
for category, probs in results.items():
    print(f"{category}: {probs.shape}")  # (batch, layers)

# Create a plotly plot showing mean probabilities for each category across layers
import plotly.graph_objects as go

# Calculate mean probabilities across batches for each category and layer
mean_probs = {category: probs.mean(dim=0) for category, probs in results.items()}

fig = go.Figure()

# Add a line for each category
for category, probs in mean_probs.items():
    fig.add_trace(
        go.Scatter(
            x=list(range(len(probs))),
            y=probs.tolist(),
            mode="lines+markers",
            name=category,
            line=dict(width=2),
            marker=dict(size=6),
        )
    )

fig.update_layout(
    title="Mean Token Probabilities Across Layers",
    xaxis_title="Layer",
    yaxis_title="Mean Probability",
    hovermode="x unified",
    template="plotly_white",
)

fig.show()

Running prompts:   0%|          | 0/1 [00:00<?, ?it/s]

target: torch.Size([2, 12])
english: torch.Size([2, 12])
format: torch.Size([2, 12])


## 9. Visualization

Finally, `nnterp` provides visualization utilities for analyzing model probabilities and prompts:

In [15]:
from nnterp.display import plot_topk_tokens, prompts_to_df

probs = logit_lens(demo_model, prompts_str[0])
# Visualize top tokens from logit lens
plot_topk_tokens(
    probs[0],
    demo_model.tokenizer,
    k=5,
    width=1000,
    height=1000,
    title="Top 5 tokens at each layer for 'The translation of 'car' in French is",
)

# Convert prompts to DataFrame for analysis
df = prompts_to_df(prompts, demo_model.tokenizer)
print("\nPrompts DataFrame:")
display(df)


Prompts DataFrame:


,0,1
prompt,The translation of 'car' in French is,The translation of 'cat' in Spanish is
target_string,"[voiture, bagnole]","[gato, minino]"
english_string,car,cat
format_string,','
target_tokens,"[vo, Ġvo, b, Ġb]","[g, Ġg, min, Ġmin]"
english_tokens,"[car, Ġcar]","[cat, Ġcat]"
format_tokens,"[', Ġ']","[', Ġ']"


# Advanced usage

Sometime, your model might not be supported yet by nnterp. In this case, you'll be able to use a `RenameConfig` to properly initialize your model.

In this section, I'll show you the steps I took to add support for the `gpt2` to `nnterp`.

###  Renaming a module not automatically renamed

Let's say that you load a `gpt2` model that is a bit special: every module is called "super_module" instead of "module".

First, let's build such a model:

In [16]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("gpt2")
for layer in model.transformer.h:
    layer.super_mlp = layer.mlp
    delattr(layer, "mlp")
    layer.super_attn = layer.attn
    delattr(layer, "attn")
model.transformer.super_h = model.transformer.h
delattr(model.transformer, "h")
# Let's keep the final layer norm as is
# model.transformer.super_ln_f = model.transformer.ln_f
# delattr(model.transformer, "ln_f")
model.super_transformer = model.transformer
delattr(model, "transformer")
print(model)

GPT2LMHeadModel(
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
  (super_transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (super_h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (super_mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (super_attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
  )
)


now if we try to use nnterp, the renaming check automatically performed will fail:

In [17]:
from nnterp import StandardizedTransformer
from traceback import print_exc

try:
    StandardizedTransformer(model)
except Exception as e:
    print_exc()

2025-10-18 06:58:37.160 | INFO     | nnterp.standardized_transformer:__init__:91 - Enforcing eager attention implementation for attention pattern tracing. The HF default would be to use sdpa if available. To use sdpa, set attn_implementation='sdpa' or None to use the HF default.
Traceback (most recent call last):
  File "/tmp/ipykernel_491885/1758275130.py", line 5, in <module>
    StandardizedTransformer(model)
  File "/mnt/nw/home/c.dumas/projects/nnterp/nnterp/standardized_transformer.py", line 128, in __init__
    self.num_layers = len(self.layers)
  File "/mnt/nw/home/c.dumas/projects/nnterp/.venv/lib/python3.10/site-packages/nnsight/intervention/envoy.py", line 1048, in __getattr__
    raise AttributeError(f"{self} has no attribute {name}")
AttributeError: GPT2LMHeadModel(
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
  (super_transformer): GPT2Model(
    (embed_tokens/wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, in

`nnterp` can't find the layers because they're located under `super_transformer`, that nnterp doesn't know about. We have 2 choices in this case:
1. Rename `super_transformer` to `model` and `super_h` to `layers` such that it matches the `model.model.layers` Llama architecture and let `nnterp` do the rest.
2. Rename `super_transformer.super_h` directly to `layers`, matching the StandardizedTransformer architecture.

Let's try the second option first. And let's not forget that we still need to rename

In order to do that we can instantiate a `StandardizedTransformer` with a `RenameConfig` with the correct aliases provided.

In [18]:
from nnterp.rename_utils import RenameConfig

rename_cfg = RenameConfig(
    layers_name="super_transformer.super_h",
    attn_name="super_attn",
    mlp_name="super_mlp",
)
try:
    StandardizedTransformer(model, rename_config=rename_cfg)
except Exception as e:
    print_exc()

2025-10-18 06:58:37.757 | INFO     | nnterp.standardized_transformer:__init__:91 - Enforcing eager attention implementation for attention pattern tracing. The HF default would be to use sdpa if available. To use sdpa, set attn_implementation='sdpa' or None to use the HF default.
Traceback (most recent call last):
  File "/tmp/ipykernel_491885/4292719532.py", line 9, in <module>
    StandardizedTransformer(model, rename_config=rename_cfg)
  File "/mnt/nw/home/c.dumas/projects/nnterp/nnterp/standardized_transformer.py", line 140, in __init__
    check_model_renaming(self, model_name, ignores, allow_dispatch)
  File "/mnt/nw/home/c.dumas/projects/nnterp/nnterp/rename_utils.py", line 680, in check_model_renaming
    raise RenamingError(
nnterp.rename_utils.RenamingError: Could not find ln_final module in GPT2LMHeadModel architecture. This means that it was not properly renamed.
Please pass the name of the ln_final module to the ln_final_rename argument.


We're still getting an error because `nnterp` doesn't find the `ln_f`. This is because `nnterp` will automatically rename the `ln_f` to `ln_final`, but fails to rename `model.ln_final` to `ln_final`. Again, we can either rename `super_transformer` to `model` or directly rename `super_transformer.ln_f` to `ln_final`.

⚠️ The code will still fail, because our "super_gpt2" model can't run its forward pass as we deleted its modules.

In [19]:
rename_cfg = RenameConfig(
    model_name="super_transformer",
    layers_name="super_h",
    attn_name="super_attn",
    mlp_name="super_mlp",
    ln_final_name="super_transformer.ln_f",
)
from transformers import AutoConfig

try:
    StandardizedTransformer(
        model, rename_config=rename_cfg, config=AutoConfig.from_pretrained("gpt2")
    )
except Exception as e:
    print_exc()

2025-10-18 06:58:38.150 | INFO     | nnterp.standardized_transformer:__init__:91 - Enforcing eager attention implementation for attention pattern tracing. The HF default would be to use sdpa if available. To use sdpa, set attn_implementation='sdpa' or None to use the HF default.
2025-10-18 06:58:38.397 | WARNING  | nnterp.utils:try_with_scan:141 - Error when trying to scan the model - using .trace() instead (which will dispatch the model)...
Traceback (most recent call last):
  File "/mnt/nw/home/c.dumas/projects/nnterp/nnterp/utils.py", line 145, in try_with_scan
    with model.trace(dummy_inputs()) as tracer:
  File "/mnt/nw/home/c.dumas/projects/nnterp/.venv/lib/python3.10/site-packages/nnsight/intervention/tracing/base.py", line 601, in __exit__
    self.backend(self)
  File "/mnt/nw/home/c.dumas/projects/nnterp/.venv/lib/python3.10/site-packages/nnsight/intervention/backends/execution.py", line 24, in __call__
    raise wrap_exception(e, tracer.info) from None
nnsight.NNsightExcep

## Adding attention probabilities support

To access the attention probabilities, `nnterp` uses the `NNsight` ability to hook on most intermediate variables of the forward pass. This is very architecture dependent, as even 2 equivalent models, if they use different names for the intermediate variables, will need different hooks.

As I'm writing this tutorial, I'm adding support for attention probabilities for `GPTJ` models.

In [ ]:
from nnterp import StandardizedTransformer

gptj = StandardizedTransformer(
    "yujiepan/gptj-tiny-random", enable_attention_probs=True
)  # In the current version of nnterp, this will work out of the box

As you can see, when you load a model,`nnterp` will automatically test if the attention probabilities hook is working and returns a tensor of shape `(batch_size, num_heads, seq_len, seq_len)` where the last dimension sums to 1. In this case, the test failed and `nnterp` logs the error.

Now let's look at the `yujiepan/gptj-tiny-random` forward pass and try to understand where are the attention probabilities computed

In [21]:
from nnterp.utils import display_source

display_source(gptj.attentions[0].source)

```py
                                 * def forward(
                                 0     self,
                                 1     hidden_states: torch.FloatTensor,
                                 2     layer_past: Optional[Cache] = None,
                                 3     attention_mask: Optional[torch.FloatTensor] = None,
                                 4     position_ids: Optional[torch.LongTensor] = None,
                                 5     head_mask: Optional[torch.FloatTensor] = None,
                                 6     use_cache: Optional[bool] = False,
                                 7     output_attentions: Optional[bool] = False,
                                 8     cache_position: Optional[torch.LongTensor] = None,
                                 9 ) -> Union[
                                10     tuple[torch.Tensor, tuple[torch.Tensor]],
                                11     Optional[tuple[torch.Tensor, tuple[torch.Tensor], tuple[torch.Tensor, ...]]],
                                12 ]:
 self_q_proj_0               -> 13     query = self.q_proj(hidden_states)
 self_k_proj_0               -> 14     key = self.k_proj(hidden_states)
 self_v_proj_0               -> 15     value = self.v_proj(hidden_states)
                                16 
 self__split_heads_0         -> 17     query = self._split_heads(query, self.num_attention_heads, self.head_dim, True)
 self__split_heads_1         -> 18     key = self._split_heads(key, self.num_attention_heads, self.head_dim, True)
 self__split_heads_2         -> 19     value = self._split_heads(value, self.num_attention_heads, self.head_dim, False)
                                20 
 is_torch_fx_proxy_0         -> 21     if is_torch_fx_proxy(position_ids) or torch.jit.is_tracing():
 torch_jit_is_tracing_0      ->  +     ...
                                22         # The logic to conditionally copy to GPU could not be traced, so we do this
                                23         # every time in the torch.fx case
 get_embed_positions_0       -> 24         embed_positions = get_embed_positions(self.embed_positions, position_ids)
                                25     else:
 self__get_embed_positions_0 -> 26         embed_positions = self._get_embed_positions(position_ids)
                                27 
 position_ids_unsqueeze_0    -> 28     repeated_position_ids = position_ids.unsqueeze(-1).repeat(1, 1, embed_positions.shape[-1])
 repeat_0                    ->  +     ...
 torch_gather_0              -> 29     sincos = torch.gather(embed_positions, 1, repeated_position_ids).to(key.dtype)
 to_0                        ->  +     ...
 torch_split_0               -> 30     sin, cos = torch.split(sincos, sincos.shape[-1] // 2, dim=-1)
                                31 
                                32     if self.rotary_dim is not None:
                                33         k_rot = key[:, :, :, : self.rotary_dim]
                                34         k_pass = key[:, :, :, self.rotary_dim :]
                                35 
                                36         q_rot = query[:, :, :, : self.rotary_dim]
                                37         q_pass = query[:, :, :, self.rotary_dim :]
                                38 
 apply_rotary_pos_emb_0      -> 39         k_rot = apply_rotary_pos_emb(k_rot, sin, cos)
 apply_rotary_pos_emb_1      -> 40         q_rot = apply_rotary_pos_emb(q_rot, sin, cos)
                                41 
 torch_cat_0                 -> 42         key = torch.cat([k_rot, k_pass], dim=-1)
 torch_cat_1                 -> 43         query = torch.cat([q_rot, q_pass], dim=-1)
                                44     else:
 apply_rotary_pos_emb_2      -> 45         key = apply_rotary_pos_emb(key, sin, cos)
 apply_rotary_pos_emb_3      -> 46         query = apply_rotary_pos_emb(query, sin, cos)
                                47 
 key_permute_0               -> 48     key = key.permute(0, 2, 1, 3)
 query_permute_0             -> 49     query = query.permute(0, 2, 1, 3)
                                50 
                                51     if layer_past is not None:
                                52         cache_kwargs = {
                                53             "sin": sin,
                                54             "cos": cos,
                                55             "partial_rotation_size": self.rotary_dim,
                                56             "cache_position": cache_position,
                                57         }
 layer_past_update_0         -> 58         key, value = layer_past.update(key, value, self.layer_idx, cache_kwargs)
                                59 
                                60     # compute self-attention: V x Softmax(QK^T)
 self__attn_0                -> 61     attn_output, attn_weights = self._attn(query, key, value, attention_mask, head_mask)
                                62 
 self__merge_heads_0         -> 63     attn_output = self._merge_heads(attn_output, self.num_attention_heads, self.head_dim)
 self_out_proj_0             -> 64     attn_output = self.out_proj(attn_output)
 self_resid_dropout_0        -> 65     attn_output = self.resid_dropout(attn_output)
                                66 
                                67     return attn_output, attn_weights
                                68 
```

Lines 60-61:
```py
                                60     # compute self-attention: V x Softmax(QK^T)
 self__attn_0                -> 61     attn_output, attn_weights = self._attn(query, key, value, attention_mask, head_mask)
 ```
⚠️ Be careful! if you set the hook here, you'll be able to successfully access the attention probabilities, but not to edit them! ⚠️

We need to check the source of `self__attn_0` to see where `attn_weights` is used. In order to access a deeper variable like this, we have to actually run the model with `trace` or `scan`. I'd advise to start with `scan` first, but switch to `trace` if you encounter an error.

In [22]:
with gptj.scan("a"):
    display_source(gptj.attentions[0].source.self__attn_0.source)

```py
                             * def _attn(
                             0     self,
                             1     query,
                             2     key,
                             3     value,
                             4     attention_mask=None,
                             5     head_mask=None,
                             6 ):
                             7     # Keep the attention weights computation in fp32 to avoid overflow issues
 query_to_0              ->  8     query = query.to(torch.float32)
 key_to_0                ->  9     key = key.to(torch.float32)
                            10 
 key_transpose_0         -> 11     attn_weights = torch.matmul(query, key.transpose(-1, -2))
 torch_matmul_0          ->  +     ...
                            12     attn_weights = attn_weights / self.scale_attn
                            13 
                            14     if attention_mask is not None:  # no matter the length, we just slice it
                            15         causal_mask = attention_mask[:, :, :, : key.shape[-2]]
                            16         attn_weights = attn_weights + causal_mask
                            17 
 nn_functional_softmax_0 -> 18     attn_weights = nn.functional.softmax(attn_weights, dim=-1)
 attn_weights_to_0       -> 19     attn_weights = attn_weights.to(value.dtype)
 self_attn_dropout_0     -> 20     attn_weights = self.attn_dropout(attn_weights)
                            21 
                            22     # Mask heads if we want to
                            23     if head_mask is not None:
                            24         attn_weights = attn_weights * head_mask
                            25 
 torch_matmul_1          -> 26     attn_output = torch.matmul(attn_weights, value)
                            27 
                            28     return attn_output, attn_weights
                            29 
```

Here, line 20-24:
```py
 self_attn_dropout_0     -> 20     attn_weights = self.attn_dropout(attn_weights)
                            21
                            22     # Mask heads if we want to
                            23     if head_mask is not None:
                            24         attn_weights = attn_weights * head_mask
```

In the current `NNsight` version, the results of operators like `*` are not hooked. But even if they were, I'd be careful to use line 24 here, as it's inside a `if` statement. Therefore, we'll use `self_attn_dropout_0` instead.

Note that we could also look at `torch_matmul_1` input and edit the value here. However, this looks less robust to me as it assumes this is the only place where `attn_weights` is used.

In [23]:
import torch as th

with gptj.scan(th.tensor([[1, 2, 3]])):
    print(
        gptj.attentions[0].source.self__attn_0.source.self_attn_dropout_0.output.shape
    )

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


torch.Size([1, 2, 3, 3])


Nice! The shape looks good. Now we can initialize our model with the right RenameConfig, and let `nnterp` run the tests for us.

To do this, we'll need to create a `AttnProbFunction` and implement the `get_attention_prob_source` method.

In [ ]:
from nnterp.rename_utils import AttnProbFunction, RenameConfig


class GPTJAttnProbFunction(AttnProbFunction):

    def get_attention_prob_source(
        self, attention_module, return_module_source: bool = False
    ):
        if return_module_source:
            # in this case, return source of the module from where the attention probabilities are computed
            return attention_module.source.self__attn_0.source
        else:
            # in this case, return the attention probabilities hook
            return attention_module.source.self__attn_0.source.self_attn_dropout_0


gptj = StandardizedTransformer(
    "yujiepan/gptj-tiny-random",
    enable_attention_probs=True,
    rename_config=RenameConfig(attn_prob_source=GPTJAttnProbFunction()),
)

with gptj.trace("Hello world!"):
    batch_size, seq_len = gptj.input_size
    attn_probs = gptj.attention_probabilities[0].save()
    print(f"attn_probs.shape: {attn_probs.shape}")
    assert attn_probs.shape == (batch_size, gptj.num_heads, seq_len, seq_len)
    gptj.attention_probabilities[0] = attn_probs / 2
    corrupt_logits = gptj.logits.save()

with gptj.trace("Hello world!"):
    clean_logits = gptj.logits.save()

assert gptj.attention_probabilities.enabled
assert not th.allclose(clean_logits, corrupt_logits)
summed_attn_probs = attn_probs.sum(dim=-1)
assert th.allclose(summed_attn_probs, th.ones_like(summed_attn_probs))

## Summary

`nnterp` provides a unified, standardized interface for working with transformer models, built on top of `nnsight`. Key features include:

1. **Standardized naming** across all transformer architectures
2. **Easy access** to layer/attention/MLP inputs and outputs
3. **Built-in methods** for common operations (steering, skipping layers, projecting to vocab)
4. **Efficient activation collection** with batching support
5. **Prompt utilities** for tracking target tokens
6. **Intervention methods** from mechanistic interpretability research
7. **Visualization tools** for analyzing model behavior

All of this while maintaining the full power and flexibility of `nnsight` under the hood!

# Appendix: `NNsight` cheatsheet

## 1) You must execute your interventions in order
In the new `NNsight` versions, it is enforced that you must access to model internals *in the same order* as the model execute them.

In [25]:
from nnterp import StandardizedTransformer
from traceback import print_exc

nnterp_gpt2 = StandardizedTransformer("gpt2")
try:
    with nnterp_gpt2.trace("My tailor is rich"):
        l2 = nnterp_gpt2.layers_output[2]
        l1 = nnterp_gpt2.layers_output[1]  # will fail! You need to collect l1 before l2
except Exception as e:
    print_exc()

2025-10-18 06:58:45.921 | INFO     | nnterp.standardized_transformer:__init__:91 - Enforcing eager attention implementation for attention pattern tracing. The HF default would be to use sdpa if available. To use sdpa, set attn_implementation='sdpa' or None to use the HF default.
/mnt/nw/home/c.dumas/projects/nnterp/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:829: UserWarning:

Can't initialize NVML

Traceback (most recent call last):
  File "/tmp/ipykernel_491885/2722117291.py", line 6, in <module>
    with nnterp_gpt2.trace("My tailor is rich"):
  File "/mnt/nw/home/c.dumas/projects/nnterp/.venv/lib/python3.10/site-packages/nnsight/intervention/tracing/base.py", line 601, in __exit__
    self.backend(self)
  File "/mnt/nw/home/c.dumas/projects/nnterp/.venv/lib/python3.10/site-packages/nnsight/intervention/backends/execution.py", line 24, in __call__
    raise wrap_exception(e, tracer.info) from None
nnsight.NNsightException: 

Traceback (most recent call last):
  File "/

## 2) Gradient computation
To compute gradients, you need to open a `.backward()` context, and save the gradients *inside it*.

In [26]:
with nnterp_gpt2.trace("My tailor is rich"):
    l1_out = nnterp_gpt2.layers_output[1]  # get l1 before accessing logits
    logits = nnterp_gpt2.output.logits
    with logits.sum().backward(
        retain_graph=True
    ):  # use retain_graph if you want to do multiple backprops
        if False:
            l1_grad = nnterp_gpt2.layers_output[1].grad.save()
            # this would fail as we'd access nnterp_gpt2.layers_output[1] after nnterp_gpt2.output
        l1_grad = l1_out.grad.save()
    with (logits.sum() ** 2).backward():
        l1_grad_2 = l1_out.grad.save()

assert not th.allclose(l1_grad, l1_grad_2)

## 3) Use tracer.stop() to save useless computations
If you're just computing activations, don't forget to call `tracer.stop()` at the end of your trace. This will stop the model from executing the rest of its computations, and save you some time, as demonstrated below (with the contribution of Claude 4 Sonnet).

In [27]:
import time
import pandas as pd

print(
    "🎭 Welcome to the Theatrical Performance Comparison! 🎭\n"
    + "=" * 60
    + "\n\n🐌 ACT I: 'The Tragedy of the Unstoppable Tracer' 🐌\nIn which our hero forgets to call tracer.stop()..."
)

start_time = time.time()
for _ in range(30):
    with nnterp_gpt2.trace(["Neel Samba", "Chris Aloha"]):
        out5 = nnterp_gpt2.layers_output[5].save()
end_time = time.time()
nostop_time = end_time - start_time

print(
    f"⏰ Duration of suffering: {nostop_time:.4f} seconds\n\n⚡ ACT II: 'The Redemption of the Stopped Tracer' ⚡\nOur hero learns the ancient art of tracer.stop()..."
)


start_time = time.time()
for _ in range(30):
    with nnterp_gpt2.trace(["Neel Samba", "Chris Aloha"]) as tracer:
        out5 = nnterp_gpt2.layers_output[5].save()
        tracer.stop()
end_time = time.time()
stop_time = end_time - start_time

print(f"⏰ Duration of enlightenment: {stop_time:.4f} seconds")

speedup = nostop_time / stop_time
time_saved = nostop_time - stop_time

# fun display
print("\n" + "=" * 60 + "\n🎉 THE GRAND RESULTS SPECTACULAR! 🎉\n" + "=" * 60)
results_df = pd.DataFrame(
    {
        "🎭 Performance Type": [
            "Without tracer.stop() 🐌",
            "With tracer.stop() ⚡",
            "Time Saved 💰",
        ],
        "⏱️ Time (seconds)": [
            f"{nostop_time:.4f}",
            f"{stop_time:.4f}",
            f"{time_saved:.4f}",
        ],
        "🎯 Rating": ["Tragic 😭", "Magnificent! 🌟", "PROFIT! 📈"],
    }
)
display(results_df)
speedup_bars = int(speedup * 10)
meter = "█" * min(speedup_bars, 48) + "░" * (50 - min(speedup_bars, 48))
print(
    f"\n🏎️ SPEEDUP METER 🏎️\n┌{'─' * 50}┐\n│{meter}│\n└{'─' * 50}┘\n   💫 COSMIC SPEEDUP: {speedup:.2f}x FASTER! 💫"
)

🎭 Welcome to the Theatrical Performance Comparison! 🎭

🐌 ACT I: 'The Tragedy of the Unstoppable Tracer' 🐌
In which our hero forgets to call tracer.stop()...
⏰ Duration of suffering: 1.8352 seconds

⚡ ACT II: 'The Redemption of the Stopped Tracer' ⚡
Our hero learns the ancient art of tracer.stop()...
⏰ Duration of enlightenment: 0.9426 seconds

🎉 THE GRAND RESULTS SPECTACULAR! 🎉


,🎭 Performance Type,⏱️ Time (seconds),🎯 Rating
0,Without tracer.stop() 🐌,1.8352,Tragic 😭
1,With tracer.stop() ⚡,0.9426,Magnificent! 🌟
2,Time Saved 💰,0.8926,PROFIT! 📈



🏎️ SPEEDUP METER 🏎️
┌──────────────────────────────────────────────────┐
│███████████████████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░│
└──────────────────────────────────────────────────┘
   💫 COSMIC SPEEDUP: 1.95x FASTER! 💫


## 4) Using NNsight builtin cache to collect activations

`NNsight 0.5` introduces a builtin way to cache activations during the forward pass. Be careful not to call `tracer.stop()` before all the module of the cache have been accessed.

The cache supports both renamed and original module names. You can access cached activations using attribute notation or dictionary keys.

**Note:** There is a bug in the current version of `NNsight` ()

In [28]:
with nnterp_gpt2.trace("Hello") as tracer:
    cache = tracer.cache(modules=[layer for layer in nnterp_gpt2.layers[::2]]).save()

# Access with renamed names using attribute notation
print(cache.model.layers[10].output)
# Or using dictionary syntax with renamed path
print(cache["model.layers.10"].output)
# Original names still work
print(cache["model.transformer.h.10"].output)

AttributeError: 'model.language_model.layers' module path was never cached. 'CacheDict' has no matching attribute.